In [3]:
import os
import random
import pandas as pd
from datetime import datetime, timedelta
import psycopg2
from psycopg2 import sql

In [2]:

###############################################
# PARTE 1: GENERAR EL CSV CON 100 REGISTROS
###############################################

csv_filename = "./data/StudentsPerformance.csv"

# Si el CSV no existe, lo creamos con datos simulados
if not os.path.isfile(csv_filename):
    # Listas de valores posibles
    genders = ["male", "female"]
    parental_educations = ["some high school", "high school", "some college", "associate's degree", "bachelor's degree", "master's degree"]
    lunches = ["standard", "free/reduced"]
    test_preparations = ["completed", "none"]
    
    # Nombres y apellidos simulados
    nombres = ["Luis", "Ana", "Carlos", "Laura", "Pedro", "Maria", "Jorge", "Sofia", "Diego", "Elena"]
    apellidos = ["Perez", "Martinez", "Sanchez", "Lopez", "Gomez", "Diaz", "Ruiz", "Ramirez", "Torres", "Flores"]
    
    # Generar 100 registros aleatorios
    data = []
    for i in range(100):
        gender = random.choice(genders)
        parental_ed = random.choice(parental_educations)
        lunch = random.choice(lunches)
        test_prep = random.choice(test_preparations)
        math_score = random.randint(0, 100)
        reading_score = random.randint(0, 100)
        writing_score = random.randint(0, 100)
        nombre = random.choice(nombres)
        apellido = random.choice(apellidos)
        # Generar una fecha de "inscripción" aleatoria en el último año
        date_added = (datetime.now() - timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")
        
        data.append({
            "gender": gender,
            "parental level of education": parental_ed,
            "lunch": lunch,
            "test preparation course": test_prep,
            "math score": math_score,
            "reading score": reading_score,
            "writing score": writing_score,
            "Nombre": nombre,
            "Apellido": apellido,
            "date_added": date_added
        })
        
    # Crear el DataFrame y guardarlo en CSV
    df_csv = pd.DataFrame(data)
    df_csv.to_csv(csv_filename, index=False)
    print(f"CSV generado: {csv_filename}")
else:
    print(f"El CSV {csv_filename} ya existe.")




CSV generado: ./data/StudentsPerformance.csv


In [4]:
###############################################
# PARTE 2: CREAR BASE DE DATOS, TABLAS E INSERTAR DATOS
###############################################

# Datos de conexión a PostgreSQL (ajusta según tu configuración)
DB_HOST = "localhost"
DB_PORT = "5432"
DB_USER = "postgres"        # Reemplaza por tu usuario
DB_PASSWORD = os.getenv('ypass_psql') # Reemplaza por tu contraseña

# 1. Conectar a la base de datos "postgres" para crear la nueva base
conn_master = psycopg2.connect(
    dbname="postgres",
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
conn_master.autocommit = True
cur_master = conn_master.cursor()

In [5]:
# 2. Conectar a la base de datos Academia (Crear base de datos antes)
conn = psycopg2.connect(
    dbname="academia",
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)
conn.autocommit = True
cur = conn.cursor()

In [5]:
# 3. Crear esquemas y tablas
cur.execute("DROP SCHEMA IF EXISTS Estudiantes CASCADE;")
cur.execute("DROP SCHEMA IF EXISTS Cursos CASCADE;")
cur.execute("CREATE SCHEMA Estudiantes;")
cur.execute("CREATE SCHEMA Cursos;")

In [6]:
# Tabla de estudiantes (Información)
create_table_estudiantes = """
CREATE TABLE Estudiantes.Informacion (
    ID SERIAL PRIMARY KEY,
    Nombre VARCHAR(50),
    Apellido VARCHAR(50),
    Gender VARCHAR(10),
    ParentalEducation VARCHAR(50),
    Lunch VARCHAR(20),
    TestPreparation VARCHAR(20),
    MathScore INT,
    ReadingScore INT,
    WritingScore INT,
    DateAdded DATE
);
"""
cur.execute(create_table_estudiantes)

# Tabla de cursos (Información) – creamos algunos cursos ficticios
create_table_cursos = """
CREATE TABLE Cursos.Informacion (
    ID SERIAL PRIMARY KEY,
    Nombre VARCHAR(100),
    Descripcion TEXT,
    FechaInicio DATE,
    FechaFin DATE
);
"""
cur.execute(create_table_cursos)

# Tabla de inscripciones: relaciona estudiantes y cursos
create_table_inscripciones = """
CREATE TABLE Estudiantes.Inscripciones (
    ID SERIAL PRIMARY KEY,
    EstudianteID INT,
    CursoID INT,
    FechaInscripcion DATE,
    FOREIGN KEY (EstudianteID) REFERENCES Estudiantes.Informacion(ID),
    FOREIGN KEY (CursoID) REFERENCES Cursos.Informacion(ID)
);
"""

cur.execute(create_table_inscripciones)
print("Tablas creadas.")


Tablas creadas.


In [7]:

# 4. Insertar datos en Estudiantes.Informacion desde el CSV generado
df_students = pd.read_csv(csv_filename)

# Insertar cada registro en la tabla de estudiantes
insert_student_query = """
INSERT INTO Estudiantes.Informacion 
(Nombre, Apellido, Gender, ParentalEducation, Lunch, TestPreparation, MathScore, ReadingScore, WritingScore, DateAdded)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

for _, row in df_students.iterrows():
    cur.execute(insert_student_query, (
        row["Nombre"],
        row["Apellido"],
        row["gender"],
        row["parental level of education"],
        row["lunch"],
        row["test preparation course"],
        row["math score"],
        row["reading score"],
        row["writing score"],
        row["date_added"]
    ))
print("Datos de estudiantes insertados.")


Datos de estudiantes insertados.


In [8]:

# 5. Insertar datos en Cursos.Informacion (cursos ficticios)
insert_course_query = """
INSERT INTO Cursos.Informacion (Nombre, Descripcion, FechaInicio, FechaFin)
VALUES (%s, %s, %s, %s);
"""
cursos = [
    ("Matematicas", "Curso de fundamentos de matematicas", "2024-09-01", "2024-12-15"),
    ("Lectura", "Curso para mejorar la comprension lectora", "2024-09-01", "2024-12-15"),
    ("Escritura", "Curso de redaccion y composicion", "2024-09-01", "2024-12-15")
]

for curso in cursos:
    cur.execute(insert_course_query, curso)
print("Datos de cursos insertados.")


Datos de cursos insertados.


In [13]:

# 6. Generar inscripciones aleatorias
cur.execute("SELECT ID FROM Estudiantes.Informacion;")

#print(cur.fetchall())

estudiantes_ids = [row[0] for row in cur.fetchall()]

print(estudiantes_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


In [14]:

cur.execute("SELECT ID FROM Cursos.Informacion;")
cursos_ids = [row[0] for row in cur.fetchall()]

insert_inscripcion_query = """
INSERT INTO Estudiantes.Inscripciones (EstudianteID, CursoID, FechaInscripcion)
VALUES (%s, %s, %s);
"""

for est_id in estudiantes_ids:
    curso_id = random.choice(cursos_ids)
    # Para demostracion, usamos una fecha fija o generada aleatoriamente
    fecha_inscripcion = datetime.strptime("2024-08-15", "%Y-%m-%d").date()
    cur.execute(insert_inscripcion_query, (est_id, curso_id, fecha_inscripcion))
print("Inscripciones generadas.")


Inscripciones generadas.


In [24]:

###############################################
# PARTE 3: EJEMPLOS DE CONSULTAS SQL (JOINs, SUBCONSULTAS, OPERADORES)
###############################################

# Ejemplo 1: INNER JOIN - Estudiantes inscritos con su curso
query_inner_join = """
SELECT E.Nombre, E.Apellido, C.Nombre AS CursoNombre, I.FechaInscripcion
FROM Estudiantes.Informacion E
INNER JOIN Estudiantes.Inscripciones I ON E.ID = I.EstudianteID
INNER JOIN Cursos.Informacion C ON I.CursoID = C.ID;
"""
cur.execute(query_inner_join)

# Obtener los datos
data = cur.fetchall()

# Convertir a DataFrame
df = pd.DataFrame(data, columns=["Nombre", "Apellido", "CursoNombre", "FechaInscripcion"])

# Mostrar el DataFrame
df

,Nombre,Apellido,CursoNombre,FechaInscripcion
0,Laura,Sanchez,Matematicas,2024-08-15
1,Laura,Lopez,Matematicas,2024-08-15
2,Jorge,Sanchez,Escritura,2024-08-15
3,Maria,Ramirez,Escritura,2024-08-15
4,Laura,Sanchez,Lectura,2024-08-15
...,...,...,...,...
95,Maria,Gomez,Matematicas,2024-08-15
96,Laura,Ruiz,Matematicas,2024-08-15
97,Jorge,Torres,Escritura,2024-08-15
98,Laura,Gomez,Lectura,2024-08-15


In [25]:
# Obtener los nombres de las columnas automáticamente
column_names = [desc[0] for desc in cur.description]

# Convertir a DataFrame
df = pd.DataFrame(data, columns=column_names)

# Mostrar el DataFrame
print(df)

   nombre apellido  cursonombre fechainscripcion
0   Laura  Sanchez  Matematicas       2024-08-15
1   Laura    Lopez  Matematicas       2024-08-15
2   Jorge  Sanchez    Escritura       2024-08-15
3   Maria  Ramirez    Escritura       2024-08-15
4   Laura  Sanchez      Lectura       2024-08-15
..    ...      ...          ...              ...
95  Maria    Gomez  Matematicas       2024-08-15
96  Laura     Ruiz  Matematicas       2024-08-15
97  Jorge   Torres    Escritura       2024-08-15
98  Laura    Gomez      Lectura       2024-08-15
99  Pedro    Gomez    Escritura       2024-08-15

[100 rows x 4 columns]


In [27]:
# Ejemplo 2: LEFT JOIN - Mostrar todos los estudiantes y sus inscripciones (si existen)

query_left_join = """
SELECT E.Nombre, E.Apellido, C.Nombre AS CursoNombre, I.FechaInscripcion
FROM Estudiantes.Informacion E
LEFT JOIN Estudiantes.Inscripciones I ON E.ID = I.EstudianteID
LEFT JOIN Cursos.Informacion C ON I.CursoID = C.ID;
"""
cur.execute(query_left_join)
left_join_df = pd.DataFrame(cur.fetchall(), columns=["Nombre", "Apellido", "CursoNombre", "FechaInscripcion"])
print("\nLEFT JOIN - Todos los estudiantes y sus inscripciones:")
print(left_join_df)


LEFT JOIN - Todos los estudiantes y sus inscripciones:
   Nombre Apellido  CursoNombre FechaInscripcion
0   Laura  Sanchez  Matematicas       2024-08-15
1   Laura    Lopez  Matematicas       2024-08-15
2   Jorge  Sanchez    Escritura       2024-08-15
3   Maria  Ramirez    Escritura       2024-08-15
4   Laura  Sanchez      Lectura       2024-08-15
..    ...      ...          ...              ...
95  Maria    Gomez  Matematicas       2024-08-15
96  Laura     Ruiz  Matematicas       2024-08-15
97  Jorge   Torres    Escritura       2024-08-15
98  Laura    Gomez      Lectura       2024-08-15
99  Pedro    Gomez    Escritura       2024-08-15

[100 rows x 4 columns]


In [28]:
# Ejemplo 3: Subconsulta - Contar estudiantes inscritos por curso
query_subconsulta = """
SELECT C.Nombre,
       (SELECT COUNT(*) 
        FROM Estudiantes.Inscripciones I 
        WHERE I.CursoID = C.ID) AS NumeroEstudiantes
FROM Cursos.Informacion C;
"""
cur.execute(query_subconsulta)
subconsulta_df = pd.DataFrame(cur.fetchall(), columns=["CursoNombre", "NumeroEstudiantes"])
print("\nSubconsulta - Numero de estudiantes por curso:")
print(subconsulta_df)


Subconsulta - Numero de estudiantes por curso:
   CursoNombre  NumeroEstudiantes
0  Matematicas                 35
1      Lectura                 29
2    Escritura                 36


In [30]:
# Ejemplo 4: Subconsulta en WHERE - Cursos con más de 30 estudiante inscrito
query_subconsulta_where = """
SELECT Nombre
FROM Cursos.Informacion
WHERE (SELECT COUNT(*) 
       FROM Estudiantes.Inscripciones I 
       WHERE I.CursoID = Cursos.Informacion.ID) > 30;
"""
cur.execute(query_subconsulta_where)
subconsulta_where_df = pd.DataFrame(cur.fetchall(), columns=["CursoNombre"])
print("\nSubconsulta en WHERE - Cursos con mas de 30 estudiante:")
print(subconsulta_where_df)



Subconsulta en WHERE - Cursos con mas de 30 estudiante:
   CursoNombre
0  Matematicas
1    Escritura


### Ejercicios adicionales con filtros de tiempo

In [32]:
# Ejercicio 1: Consultar estudiantes inscritos antes de una fecha específica
query_1 = """
SELECT 
    E.Nombre, 
    E.Apellido, 
    I.FechaInscripcion
FROM Estudiantes.Informacion E
JOIN Estudiantes.Inscripciones I ON E.ID = I.EstudianteID
WHERE I.FechaInscripcion < '2025-01-01';
"""

cur.execute(query_1)
df_1 = pd.DataFrame(cur.fetchall(), columns=['Nombre', 'Apellido', 'FechaInscripcion'])
print("\nEstudiantes inscritos antes de '2025-01-01':")
print(df_1)


Estudiantes inscritos antes de '2025-01-01':
   Nombre Apellido FechaInscripcion
0   Laura  Sanchez       2024-08-15
1   Laura    Lopez       2024-08-15
2   Jorge  Sanchez       2024-08-15
3   Maria  Ramirez       2024-08-15
4   Laura  Sanchez       2024-08-15
..    ...      ...              ...
95  Maria    Gomez       2024-08-15
96  Laura     Ruiz       2024-08-15
97  Jorge   Torres       2024-08-15
98  Laura    Gomez       2024-08-15
99  Pedro    Gomez       2024-08-15

[100 rows x 3 columns]


In [34]:
# Ejercicio 2: Consultar cursos con inscripciones en un rango de fechas
query_2 = """
SELECT 
    C.Nombre AS CursoNombre, 
    I.FechaInscripcion
FROM Cursos.Informacion C
JOIN Estudiantes.Inscripciones I ON C.ID = I.CursoID
WHERE I.FechaInscripcion BETWEEN '2023-01-01' AND '2025-01-01';
"""
cur.execute(query_2)
df_2 = pd.DataFrame(cur.fetchall(), columns=['CursoNombre', 'FechaInscripcion'])
print("\nCursos con inscripciones entre '2023-01-01' y '2023-12-31':")
print(df_2)



Cursos con inscripciones entre '2023-01-01' y '2023-12-31':
    CursoNombre FechaInscripcion
0   Matematicas       2024-08-15
1   Matematicas       2024-08-15
2     Escritura       2024-08-15
3     Escritura       2024-08-15
4       Lectura       2024-08-15
..          ...              ...
95  Matematicas       2024-08-15
96  Matematicas       2024-08-15
97    Escritura       2024-08-15
98      Lectura       2024-08-15
99    Escritura       2024-08-15

[100 rows x 2 columns]


In [6]:
###############################################
# PARTE 4: CERRAR CONEXIÓN
###############################################
cur.close()
conn.close()
print("\nConexión cerrada.")


Conexión cerrada.
